In [3]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
from pdfAccess import PdfAccess

ValueError: pyarrow.lib.NativeFile size changed, may indicate binary incompatibility. Expected 104 from C header, got 96 from PyObject

In [ ]:
dataset = PdfAccess().getDataSet()

In [ ]:
df = []
for data in dataset:
    for q in data['questions']:
        df.append({'question': q, 'answer': data['answer']})

In [ ]:
df[1]

In [ ]:
# Model
base_model = "microsoft/phi-2"
new_model = "phi-2-medquad"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

In [ ]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map={"": 0}
)

model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

In [ ]:
sentences = sent_tokenize(pdf_text)

In [ ]:
sentences

In [ ]:
def extract_questions_and_answers(sentences):
    questions = []
    answers = []
    
    for sentence in sentences:
        print(sentence)
        # Check if the sentence ends with "?"
        if sentence.strip().endswith('?'):
            questions.append(sentence)
        else:
            answers.append(sentence)

    return questions, answers

In [ ]:
pdf_questions, pdf_answers = extract_questions_and_answers(sentences)

In [ ]:
pdf_questions[2], pdf_answers[2]

In [ ]:
quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True, load_in_8bit=True,)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype=torch.float32,
    device_map='auto',
    quantization_config=quantization_config, low_cpu_mem_usage=True
)